In [2]:
import tflite_runtime.interpreter as tflite
import cv2
import numpy as np
import os

In [9]:
interpreter = tflite.Interpreter(model_path='/home/suraj/tensorflow/Notebooks/Untitled Folder/face_mask_detection.tflite')
interpreter.allocate_tensors()

In [10]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [11]:
print(input_details,'\n\n', output_details)

[{'name': 'data_1', 'index': 70, 'shape': array([  1, 260, 260,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}}] 

 [{'name': 'loc_branch_concat_1/concat', 'index': 111, 'shape': array([   1, 5972,    4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}}, {'name': 'cls_branch_concat_1/concat', 'index': 45, 'shape': array([   1, 5972,    2], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}}]


In [12]:
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

In [13]:
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

In [14]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[[-0.4820956   0.6323688  -2.14772    -1.0640969 ]
  [-0.01728516  1.4262571  -0.50420713  0.6227364 ]
  [-0.9657819   0.56529486 -1.6605849  -2.0489366 ]
  ...
  [-1.6603458  -2.0319223  -0.9463796  -0.35701203]
  [-1.9181842  -1.7372411   0.12240043 -1.0256536 ]
  [-2.2860909  -1.6143186   0.91142505 -1.8157928 ]]]


In [15]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
label_dict = {0:'MASK', 1:"NO MASK"}
color_dict = {0:(0,255,0), 1:(0,0,255)}

In [16]:
label = np.argmax(output_data, axis=1)[0]

In [17]:
from time import sleep

source= cv2.VideoCapture(0)
sleep(2)

img_size=260
while True:
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray, 1.3, 5)
    
    for x, y, w ,h in faces:
        face_img = gray[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (img_size,img_size))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, input_shape)
        reshaped = np.float32(reshaped)
        interpreter.set_tensor(input_details[0]['index'], reshaped)
        interpreter.invoke()
        result = interpreter.get_tensor(output_details[0]['index'])
        
#         label = np.argmax(result, axis=1)[0]
        
#         cv2.rectangle(img,(x,y),(x+w,y+h), color_dict[label], 2)
#         cv2.rectangle(img,(x,y-4),(x+w,y), color_dict[label], -1)
#         cv2.putText(img, label_dict[label], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
    
    cv2.imshow("LIVE", result)
    key = cv2.waitKey(1)
    
    if (key==27):
        break
    
cv2.destroyAllWindows()
source.release()

ValueError: cannot reshape array of size 67600 into shape (1,260,260,3)

In [22]:
input_details

[{'name': 'input_2',
  'index': 0,
  'shape': array([  1, 100, 100,   1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0}}]

In [23]:
input_shape

array([  1, 100, 100,   1], dtype=int32)